# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [47]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [48]:
# read in csv
filepath = 'weather_data.csv'

df = pd.read_csv(filepath)
df

,City,Country,Latitude,Longitude,Date,Wind_Speed,Pressure,Humidity,Temperature,Cloudiness
0,Upernavik,GL,72.7868,-56.1549,1652912472,4.01,1017,85,-2.05,7
1,Barrow,US,71.2906,-156.7887,1652912305,8.75,1025,79,-7.99,100
2,Nantucket,US,41.2835,-70.0995,1652912474,7.20,1012,48,17.06,0
3,Mahébourg,MU,-20.4081,57.7000,1652912198,5.14,1019,69,24.16,75
4,Virginia Beach,US,36.8529,-75.9780,1652912301,3.60,1015,56,20.43,0
...,...,...,...,...,...,...,...,...,...,...
589,Robinsonville,US,31.4677,-97.1147,1652913306,9.77,1009,34,34.81,0
590,La Ronge,CA,55.1001,-105.2842,1652913308,5.14,1009,29,15.14,75
591,Yablonovo,RU,50.8523,37.3574,1652913309,4.67,1015,92,6.02,8
592,Half Moon Bay,US,37.4636,-122.4286,1652913311,10.80,1017,50,23.52,0


In [49]:
df.head()

,City,Country,Latitude,Longitude,Date,Wind_Speed,Pressure,Humidity,Temperature,Cloudiness
0,Upernavik,GL,72.7868,-56.1549,1652912472,4.01,1017,85,-2.05,7
1,Barrow,US,71.2906,-156.7887,1652912305,8.75,1025,79,-7.99,100
2,Nantucket,US,41.2835,-70.0995,1652912474,7.20,1012,48,17.06,0
3,Mahébourg,MU,-20.4081,57.7000,1652912198,5.14,1019,69,24.16,75
4,Virginia Beach,US,36.8529,-75.9780,1652912301,3.60,1015,56,20.43,0


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594 entries, 0 to 593
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   City         594 non-null    object 
 1   Country      590 non-null    object 
 2   Latitude     594 non-null    float64
 3   Longitude    594 non-null    float64
 4   Date         594 non-null    int64  
 5   Wind_Speed   594 non-null    float64
 6   Pressure     594 non-null    int64  
 7   Humidity     594 non-null    int64  
 8   Temperature  594 non-null    float64
 9   Cloudiness   594 non-null    int64  
dtypes: float64(4), int64(4), object(2)
memory usage: 46.5+ KB


In [51]:
# drop NA values
df2 = df.dropna().reset_index(drop = True)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590 entries, 0 to 589
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   City         590 non-null    object 
 1   Country      590 non-null    object 
 2   Latitude     590 non-null    float64
 3   Longitude    590 non-null    float64
 4   Date         590 non-null    int64  
 5   Wind_Speed   590 non-null    float64
 6   Pressure     590 non-null    int64  
 7   Humidity     590 non-null    int64  
 8   Temperature  590 non-null    float64
 9   Cloudiness   590 non-null    int64  
dtypes: float64(4), int64(4), object(2)
memory usage: 46.2+ KB


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [52]:
# From Week 6 Day 3 Activity 7 Prof Booth
# Configure gmaps

gmaps.configure(api_key=g_key)

In [67]:
locations = df2.loc[:, ["Latitude", "Longitude"]]

fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(
    locations, weights=df2['Humidity'],
    max_intensity=125, point_radius=20
)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [69]:
# Create Masks for the following critera: temp between 70 and 80, wind less than 10 mph, 0 cloudiness

mask = (df2.Temperature < 80) & (df2.Temperature > 70) & (df2.Wind_Speed < 10) & (df.Cloudiness == 0)
df3 = df2.loc[mask]
df3.head()

,City,Country,Latitude,Longitude,Date,Wind_Speed,Pressure,Humidity,Temperature,Cloudiness


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [70]:
hotels = []
ratings = []
addresses = []
city = []
country = []
lat = []
lng = []

# Loop through dataframe to grab lat and long for locations and google api request
# Prof Booth
for _, row in df3.iterrows():
    print(row.City)

    # grab lat and long and storing as variable
    target_coordinates = f"{row.Latitude}, {row.Longitude}"
    target_search = ""

    # set vaariables and params for API request
    target_radius = 5000
    target_type = "lodging"
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # API request
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    response = requests.get(url, params = params)

    data = response.json()

    hotel = data["results"][0]["name"]
    address = data["results"][0]["vicinity"]
    rating = data["results"][0]["rating"]
    
    # put it all in a list
    hotels.append(hotel)
    ratings.append(rating)
    addresses.append(address)
    city.append(row.City)
    country.append(row.Country)
    lat.append(row.Latitude)
    lng.append(row.Longitude)

In [71]:
# Create the hotel_df DataFrame

hotel_df = pd.DataFrame()
hotel_df['Hotel'] = hotels
hotel_df['Address'] = addresses
hotel_df['Rating'] = ratings
hotel_df['City'] = city
hotel_df['Country'] = country
hotel_df['Latitude'] = lat
hotel_df['Longitude'] = lng

hotel_df

,Hotel,Address,Rating,City,Country,Latitude,Longitude


In [73]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [77]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
heatmap_layer = gmaps.heatmap_layer(
    locations, weights=df2['Humidity'],
    max_intensity=125, point_radius=20
)

fig = gmaps.figure()

# add layers
fig.add_layer(marker_layer)
fig.add_layer(heatmap_layer)

# Display figure
fig

ValueError: weights must be of the same length as locations or None